In [2]:
import os
import pickle
import numpy as np
import time

from gensim.models import Word2Vec
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, hamming_loss, f1_score
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical

from models import get_rnn_model
from cm import multilabel_confusion_matrix
from data_process import get_embedding_matrix, data_generator, get_all_notes_labels, get_features, get_targets, get_gold_label_targets

# Customized Evaluation for keras model
class CustomEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = list(validation_data)

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = []
            for x in self.X_val:
                y = np.squeeze(self.model.predict_on_batch(x))
                y_pred.append(y)
            y_pred = np.concatenate(y_pred)
            y_pred_ham = y_pred > 0.5
            y_val = np.concatenate(self.y_val)
            roc = roc_auc_score(y_val, y_pred, average='micro')
            loss = log_loss(y_val, y_pred)
            ham = hamming_loss(y_val, y_pred_ham)
            sub = accuracy_score(y_val, y_pred_ham)
            f1 = f1_score(y_val, y_pred_ham, average='micro')
            print("Adiitional val metrics: - ROC-AUC: %.6f - Log-Loss: %.6f - Hamming-Loss: %.6f - Subset-Accuracy: %.6f - F1-Score: %.6f" % (roc, loss, ham, sub, f1))
            
def model_train(param, 
                notes_train, 
                labels_train, 
                up_notes_train, 
                up_labels_train, 
                gold_labels_train, 
                notes_test, 
                labels_test, 
                gold_labels_test,
                results_file,
                verbose=1):
    
    print('*'*80)
    print("Parameters (note: embed_size*10, latent_dim*64):\n", param)
    print('*'*80)
    
    start_time = time.time()
    
    # assign parameters
    up = int(param['up'])
    window_size = int(param['window_size'])
    embed_size = int(param['embed_size'] * 10)
    latent_dim = int(param['latent_dim'] * 64)
    dropout_rate = param['dropout_rate']
    epochs = param['epochs']
    category = param['category']
    max_features = 60000 #param['max_features']
    train_embed = True #param['train_embed']
    model_type = 'CuDNNLSTM' #param['model_type']
    
    # upsampling
    if up > 0:
        if verbose != 0: print('upsampling for %d times...' % (up))
        notes_train = [note + up * up_note for note, up_note in zip(notes_train, up_notes_train)]
        labels_train = [label + up * up_label for label, up_label in zip(labels_train, up_labels_train)]
    notes = notes_train + notes_test
    labels = labels_train + labels_test
    gold_labels = gold_labels_train + gold_labels_test
    
    # prepare features
    X_train_seq, X_test_seq, word_index = get_features(max_features, notes_train, notes_test, verbose=1)
    nb_words = min(max_features, len(word_index))

    # prepare embedding matrix
    if train_embed:
        if verbose != 0: print('preparing pretrained embedding matrix ...')
        w2v = Word2Vec(notes, size=embed_size, window=window_size, min_count=1, workers=4)
        embedding_index = dict(zip(w2v.wv.index2word, w2v.wv.vectors))
        embedding_matrix = get_embedding_matrix(embedding_index=embedding_index, 
                                                word_index=word_index, 
                                                max_features=max_features, 
                                                embed_size=embed_size)
        
    # prepare targets
    Y_train, Y_test, mlb, num_labels = get_targets(labels_train, labels_test, category, verbose=1)  

    # get rnn model
    model = get_rnn_model(nb_words=nb_words, 
                          num_labels=num_labels, 
                          embed_size=embed_size, 
                          latent_dim=latent_dim, 
                          model_type=model_type, 
                          embedding_matrix=embedding_matrix, 
                          dropout=dropout_rate, 
                          train_embed=train_embed)
    if verbose != 0: 
        print('model summary:')
        print(model.summary())
    
    # model compiling
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # model training
    if verbose != 0: print('\ntraining model ...')
    custevl = CustomEvaluation(validation_data=(X_test_seq, Y_test), interval=1)
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0005, patience=2, verbose=0, mode='auto')
    train_gen = data_generator(X_train_seq, Y_train)
    test_gen = data_generator(X_test_seq, Y_test)
    v = 1 if verbose != 0 else 0  
    hist = model.fit_generator(train_gen,
                                steps_per_epoch=len(Y_train),
                                epochs=epochs,
                                validation_data=test_gen,
                                validation_steps=len(Y_test),
                                callbacks=[custevl, earlystop],
                                verbose=v)

    # prediction of test data
    if verbose != 0: print('predicting test data ...')
    Y_pred = []
    for x in X_test_seq:
        x = np.array(x).reshape((1,-1))
        y_pred = np.squeeze(model.predict_on_batch(x))
        Y_pred.append(y_pred)
    Y_pred_concat = np.concatenate(Y_pred)
    Y_val = np.concatenate(Y_test)

    # confusion matrix 
    if verbose == 2: 
        cm = multilabel_confusion_matrix(Y_val, np.where(Y_pred_concat > 0.5, 1, 0))
        for i, j in zip(cm, mlb.classes_):
            print(j+':\n', i,'\n')

    # prepare gold label targets
    Y_gold_test, Y_gold_pred, gmlb = get_gold_label_targets(Y_pred, gold_labels, gold_labels_test, mlb, category=category, verbose=1) 

    # f1 scores for gold label
    f1 = f1_score(Y_gold_test, Y_gold_pred, average='micro')
    print('\nF1 Scores for global labels:\nALL (average="micro"):', f1)
    
    # confusion matrix for gold label
    if verbose == 2: 
        gcm = multilabel_confusion_matrix(np.concatenate(Y_gold_test), np.concatenate(Y_gold_pred))
        for i, j in zip(gcm, gmlb.classes_):
            print(j+':\n', i,'\n')
    
    # f1 score list
    if verbose == 2: 
        f1_all = f1_score(Y_gold_test, Y_gold_pred, average=None)
        for i, j in zip(f1_all, gmlb.classes_):
            print(j+': '+str(i))
    
    print('\n')
    
    elapsed_time = time.time() - start_time
    
    # save results
    with open(results_file,"a") as f:
        f.write("Parameters (note: embed_size*10, ltent_dim*64):\n" + str(param))
        f.write('\nF1 Scores for global labels(average="micro"): %.3f; Running time: %.1f\n' % (f1, elapsed_time))
          
    return f1

def bayes_opt(space):
    
    results_file = "opt_results_" + time.strftime("%Y%m%d") + ".txt"
    
    param = {
            'up': space[0],               # Times of upsampling for training data
            'window_size': space[1],                # Window size for word2vec
            'embed_size': space[2],                # Length of the vector that we willl get from the embedding layer
            'latent_dim': space[3],               # Hidden layers dimension 
            'dropout_rate': space[4],             # Rate of the dropout layers
            'epochs': space[5],               # Max num of vocabulary
            'category': space[6]}                 # Number of epochs
            #'max_features': space[0],              # Is categoty labels
            #'train_embed': space[0],               # Using pre-made embedidng matrix as weight
            #'model_type': space[0]
            #}
                
    f1 = model_train(param, 
                    notes_train, 
                    labels_train, 
                    up_notes_train, 
                    up_labels_train, 
                    gold_labels_train, 
                    notes_test, 
                    labels_test, 
                    gold_labels_test,
                    results_file=results_file)
    
    return (-f1)    

if __name__ == "__main__":
    
    # loading data 
    if os.path.exists('loaded_data.dat'):
        
        with open('loaded_data.dat','rb') as f:
            notes_train = pickle.load(f)
            labels_train = pickle.load(f)
            up_notes_train = pickle.load(f)
            up_labels_train = pickle.load(f)
            gold_labels_train = pickle.load(f)
            notes_test = pickle.load(f)
            labels_test = pickle.load(f)
            gold_labels_test = pickle.load(f)
            
    else:
        
        notes_train_1, labels_train_1, up_notes_train_1, up_labels_train_1, gold_labels_train_1 = get_all_notes_labels('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set1') 
        notes_train_2, labels_train_2, up_notes_train_2, up_labels_train_2, gold_labels_train_2 = get_all_notes_labels('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set2') 

        notes_train = notes_train_1 + notes_train_2
        labels_train = labels_train_1 + labels_train_2
        up_notes_train = up_notes_train_1 + up_notes_train_2
        up_labels_train = up_labels_train_1 + up_labels_train_2
        gold_labels_train = gold_labels_train_1 + gold_labels_train_2

        notes_test, labels_test, _1, _2, gold_labels_test = get_all_notes_labels('/host_home/data/i2b2/2014/testing/testing-RiskFactors-Complete')

        with open('loaded_data.dat','wb') as f:
            pickle.dump(notes_train, f)
            pickle.dump(labels_train, f)
            pickle.dump(up_notes_train, f)
            pickle.dump(up_labels_train, f)
            pickle.dump(gold_labels_train, f)
            pickle.dump(notes_test, f)
            pickle.dump(labels_test, f)
            pickle.dump(gold_labels_test, f)

    # loading parameters space
    param = {'up': 5, 'window_size': 3, 'embed_size': 5, 'latent_dim': 5, 'dropout_rate': 0.0, 'epochs': 20, 'category': None}
    
    model_train(param, 
                notes_train, 
                labels_train, 
                up_notes_train, 
                up_labels_train, 
                gold_labels_train, 
                notes_test, 
                labels_test, 
                gold_labels_test,
                results_file=results_file)

Iteration No: 1 started. Evaluating function at random point.
********************************************************************************
Parameters (note: embed_size*10, latent_dim*64):
 {'up': 11, 'window_size': 4, 'embed_size': 4, 'latent_dim': 5, 'dropout_rate': 0.37619637288650404, 'epochs': 21, 'category': None}
********************************************************************************
upsampling for 11 times...
preparing features ...
preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_101 (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_101 (Embedding)    (None, None, 40)          1799360   
_________________________________________________________________
bidirectional_101 (Bidirecti (None, None, 640)         926720    
___

Epoch 1/20
790/790 [==============================] - 250s 317ms/step - loss: 0.0220 - acc: 0.9949 - val_loss: 0.0064 - val_acc: 0.9988
Adiitional val metrics: - ROC-AUC: 0.956992 - Log-Loss: 3.342014 - Hamming-Loss: 0.006210 - Subset-Accuracy: 0.647712 - F1-Score: 0.690825
Epoch 2/20
790/790 [==============================] - 216s 274ms/step - loss: 0.0113 - acc: 0.9973 - val_loss: 0.0047 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.976304 - Log-Loss: 2.692041 - Hamming-Loss: 0.003769 - Subset-Accuracy: 0.820296 - F1-Score: 0.824044
Epoch 3/20
790/790 [==============================] - 217s 275ms/step - loss: 0.0088 - acc: 0.9977 - val_loss: 0.0041 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.981220 - Log-Loss: 2.331304 - Hamming-Loss: 0.002903 - Subset-Accuracy: 0.841974 - F1-Score: 0.863321
Epoch 4/20
790/790 [==============================] - 216s 274ms/step - loss: 0.0073 - acc: 0.9980 - val_loss: 0.0039 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.98

preparing features ...
preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_106 (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_106 (Embedding)    (None, None, 40)          1799360   
_________________________________________________________________
bidirectional_106 (Bidirecti (None, None, 384)         359424    
_________________________________________________________________
spatial_dropout1d_106 (Spati (None, None, 384)         0         
_________________________________________________________________
dense_211 (Dense)            (None, None, 300)         115500    
_________________________________________________________________
dense_212 (Dense)            (None, None, 97)          29197     
Total params: 2,303,481
Trainable params

790/790 [==============================] - 112s 142ms/step - loss: 0.0109 - acc: 0.9974 - val_loss: 0.0048 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.980801 - Log-Loss: 2.497474 - Hamming-Loss: 0.002105 - Subset-Accuracy: 0.879252 - F1-Score: 0.898650
Epoch 3/20
790/790 [==============================] - 112s 142ms/step - loss: 0.0083 - acc: 0.9979 - val_loss: 0.0041 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.984756 - Log-Loss: 2.072491 - Hamming-Loss: 0.001819 - Subset-Accuracy: 0.901197 - F1-Score: 0.913135
Epoch 4/20
790/790 [==============================] - 112s 142ms/step - loss: 0.0067 - acc: 0.9981 - val_loss: 0.0039 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.986748 - Log-Loss: 1.808071 - Hamming-Loss: 0.001758 - Subset-Accuracy: 0.901736 - F1-Score: 0.915902
Epoch 5/20
790/790 [==============================] - 111s 141ms/step - loss: 0.0057 - acc: 0.9983 - val_loss: 0.0039 - val_acc: 0.9991
Adiitional val metrics: - ROC-AUC: 0.987762 - Log-

preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_111 (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_111 (Embedding)    (None, None, 50)          2249200   
_________________________________________________________________
bidirectional_111 (Bidirecti (None, None, 640)         952320    
_________________________________________________________________
spatial_dropout1d_111 (Spati (None, None, 640)         0         
_________________________________________________________________
dense_221 (Dense)            (None, None, 300)         192300    
_________________________________________________________________
dense_222 (Dense)            (None, None, 97)          29197     
Total params: 3,423,017
Trainable params: 3,423,017
Non-trainab

...

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_166 (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_166 (Embedding)    (None, None, 50)          2249200   
_________________________________________________________________
bidirectional_166 (Bidirecti (None, None, 384)         374784    
_________________________________________________________________
spatial_dropout1d_166 (Spati (None, None, 384)         0         
_________________________________________________________________
dense_331 (Dense)            (None, None, 300)         115500    
_________________________________________________________________
dense_332 (Dense)            (None, None, 97)          29197     
Total params: 2,768,681
Trainable params: 2,768,681
Non-trainab

790/790 [==============================] - 152s 193ms/step - loss: 0.0115 - acc: 0.9972 - val_loss: 0.0048 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.973871 - Log-Loss: 2.596532 - Hamming-Loss: 0.002780 - Subset-Accuracy: 0.851881 - F1-Score: 0.868624
Epoch 3/20
790/790 [==============================] - 153s 193ms/step - loss: 0.0089 - acc: 0.9977 - val_loss: 0.0042 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.980827 - Log-Loss: 2.169802 - Hamming-Loss: 0.001987 - Subset-Accuracy: 0.892003 - F1-Score: 0.905603
Epoch 4/20
790/790 [==============================] - 153s 193ms/step - loss: 0.0072 - acc: 0.9980 - val_loss: 0.0039 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.983317 - Log-Loss: 1.897188 - Hamming-Loss: 0.001808 - Subset-Accuracy: 0.898429 - F1-Score: 0.913795
Epoch 5/20
790/790 [==============================] - 152s 192ms/step - loss: 0.0060 - acc: 0.9983 - val_loss: 0.0038 - val_acc: 0.9991
Adiitional val metrics: - ROC-AUC: 0.984331 - Log-

preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_171 (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_171 (Embedding)    (None, None, 50)          2249200   
_________________________________________________________________
bidirectional_171 (Bidirecti (None, None, 640)         952320    
_________________________________________________________________
spatial_dropout1d_171 (Spati (None, None, 640)         0         
_________________________________________________________________
dense_341 (Dense)            (None, None, 300)         192300    
_________________________________________________________________
dense_342 (Dense)            (None, None, 97)          29197     
Total params: 3,423,017
Trainable params: 3,423,017
Non-trainab

ResourceExhaustedError: OOM when allocating tensor with shape[1,19556,320] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node bidirectional_171/ReverseV2_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_170/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
